In [1]:
import dicom2nifti
import os
from pathlib import Path
import os
import numpy as np
import matplotlib.pyplot as plt
import imageio
import glob
import nibabel as nib
from itertools import chain
import SimpleITK as sitk
import imagej
import nrrd
from sklearn.mixture import GaussianMixture
from ipywidgets import interact, fixed
from IPython.display import display

import SimpleITK as sitk
import imagej
# from PIL import Image
import pydicom as dicom
import os
import numpy
from matplotlib import pyplot, cm
from rt_utils import RTStructBuilder

In [2]:
import dicom2nifti
import os
from pathlib import Path
import os
import numpy as np
import matplotlib.pyplot as plt
import imageio
import glob
import nibabel as nib
from itertools import chain
from util.imgsitk import *
import SimpleITK as sitk
import imagej
from ipywidgets import interact, fixed
import itk
from IPython.display import clear_output
from util.spineSeg import *
from util.spineSegC import *
from lungmask import mask
from rt_utils import RTStructBuilder
import matplotlib.pyplot as plt
from utl.pytorch_morph import *

In [3]:
def display_images_with_alpha_numpy(image_z, alpha, fixed, moving):
    img = (1.0 - alpha)*fixed[image_z,:,:] + alpha*moving[image_z,:,:] 
    plt.figure(figsize=(16,9))
    plt.imshow(img,cmap=plt.cm.Greys_r);
    # x = plt.ginput(3)
    # print(x)
    # plt.ginput()
    # plt.axis('off')
    plt.show()

def win_scale(data, wl, ww):
    """
    Scale pixel intensity data using specified window level, width, and intensity range.
    """
    datatype=data.dtype
    image=sitk.GetImageFromArray(data)

    resacleFilter = sitk.IntensityWindowingImageFilter()
    resacleFilter.SetWindowMaximum(wl+ww/2)
    resacleFilter.SetWindowMinimum(wl-ww/2)
    resacleFilter.SetOutputMaximum(1)
    resacleFilter.SetOutputMinimum(0)

    out = resacleFilter.Execute(image)
    
    data_new=sitk.GetArrayFromImage(out)

    return data_new.astype(datatype)

def scale(x, out_range=(-1, 1)):
    domain = np.min(x), np.max(x)
    y = (x - (domain[1] + domain[0]) / 2) / (domain[1] - domain[0])
    return y * (out_range[1] - out_range[0]) + (out_range[1] + out_range[0]) / 2


class Imshow3D:
    def __init__(self,fig, ax, vol):

        self.fig=fig
        self.connect()
        self.ax = ax
        ax.set_title('use scroll wheel to navigate images')

        self.vol = vol
        self.vol_bk = vol

        self.maxvol = vol.max()
        self.minvol = vol.min()

        rows, cols, self.slices = vol.shape
        self.ind = self.slices//2


        self.maxlevel=450
        self.minlevel=-350
        self.im = ax.imshow(self.vol[:, :, self.ind], cmap='gray')
        

        self.sliders=widgets.IntSlider(description='Slice Number',value=self.ind ,min=0, max=self.slices, step=1,)
        self.sliderl=widgets.IntSlider(description='Level',value=50 ,min=self.minvol, max=self.maxvol, step=1,)
        self.sliderw=widgets.IntSlider(description='Window',value=400 ,min=1, max=self.maxvol//2, step=1,)
        # self.sliders =self.ind
        # l = widgets
        # ((sliders1, 'value')
        self.sliders.observe(self.handle_slider_change, names='value')

        self.sliderl.observe(self.handle_level_window, names='value')
        self.sliderw.observe(self.handle_level_window, names='value')

        display(self.sliders,self.sliderl,self.sliderw)
        self.update()

    def connect(self):
        
        """Connect to all the events we need."""
        self.cidpress = self.fig.canvas.mpl_connect('button_press_event', self.on_press)

        self.cidrelease = self.fig.canvas.mpl_connect('button_release_event', self.on_release)

        self.cidmotion = self.fig.canvas.mpl_connect('motion_notify_event', self.on_motion)

        self.onscroll=self.fig.canvas.mpl_connect('motion_notify_event', self.on_scroll)

        self.onkey=self.fig.canvas.mpl_connect('key_press_event', self.on_key)

    def on_scroll(self, event):
        # print("%s %s" % (event.button, event.step))
        # if event.button == 'up' :
        #     self.ind = (self.ind + 1) % self.slices
        # elif event.button == 'down' :
        #     self.ind = (self.ind - 1) % self.slices
        
        # self.update()
        True


        # if event.motion == 'up':
        #     self.ind = (self.ind + 1) % self.slices

    def on_press(self, event):
        True

    def on_release(self, event):
        True

    def on_motion(self, event):
        True

    def on_key (self, event):

        if event.key == 'up':
            self.ind = (self.ind + 1) % self.slices
        elif event.key == 'down':
            self.ind = (self.ind - 1) % self.slices
        # self.fig.canvas.draw()
        self.update()

    # def previous_slice(self,ax):
    #     """Go to the previous slice."""
    #     volume = ax.volume
    #     ax.index = (ax.index - 1) % volume.shape[0]  # wrap around using %
    #     self.ax.images[0].set_array(volume[ax.index])

    # def next_slice(self,ax):
    #     """Go to the next slice."""
    #     volume = ax.volume
    #     ax.index = (ax.index + 1) % volume.shape[0]
    #     self.ax.images[0].set_array(volume[ax.index])
    
    def handle_slider_change(self, change):
        True
        # print(self.sliders.value)
        self.ind=self.sliders.value
        self.update()


    def handle_level_window(self, change):
        """
        Function to display an image slice
        Input is a numpy 2D array
        """
        self.vol=self.vol_bk

        level = self.sliderl.value
        window = self.sliderw.value
        self.maxlevel = level + window/2
        self.minlevel = level - window/2

        if self.maxvol<self.maxlevel:
            self.maxlevel=self.maxvol
        if self.minvol>self.minlevel:
            self.minlevel=self.minvol

            



        self.update()


    def update(self):
        self.sliders.value=self.ind
        self.im.set_data(self.vol[:, :, self.ind])
        self.im.set_clim(vmin=self.minlevel,vmax=self.maxlevel)
        self.ax.set_ylabel('slice %s' % self.ind)
        plt.autoscale()
        self.im.axes.figure.canvas.draw()
        # print(self.sliders.value)





def rescale(img_data1q,stc):
    sz=img_data1q.shape
    img_data1qt=torch.from_numpy(img_data1q)
    astc=np.abs(np.round(stc)).astype(int)
    istc=tuple(np.round(-stc).astype(int))

    img_data1qt=torch.nn.functional.pad(img_data1qt.unsqueeze(0).unsqueeze(0),(0,astc[2], 0,astc[1], 0,astc[0]),mode='replicate')
    img_data1qt=torch.roll(img_data1qt, shifts=istc, dims=(2,3,4) )
    img_data1qt=img_data1qt[:,:,0:sz[0],0:sz[1],0:sz[2]].squeeze(0).squeeze(0).numpy()
    return img_data1qt

In [4]:

import astra
import torch
import torchvision.transforms as transforms

CTTYPE='CT1'



data_folder="./TestDataNii/"
list_CT=Path(data_folder).glob(('**/*_IMG_' + CTTYPE + '.nii.gz'))
list_STR=Path(data_folder).glob(('**/*_STR_' + CTTYPE + '.nii.gz'))

DataInfoTuple=[]
DataInfoTupleSTR=[]
n=0
for i in list_CT:
    DataInfoTuple.append(i)
    n=n+1
for i in list_STR:
    DataInfoTupleSTR.append(i)
    n=n+1


# for i,j in DataInfoTuple:
for kkk in range(0,2):

    i=DataInfoTuple[kkk]
    j=DataInfoTupleSTR[kkk]

    s=str(i).find('\P')
    e=str(i).find('_IMG')
    casenum=int(str(i)[s+2:e])
    print(['Case Number is ' + str(casenum)])
    img_data = nib.load(i).get_fdata()

    img_data1=np.moveaxis(img_data,2,0)
    img_data1=np.moveaxis(img_data1,1,2)
    img_data1=np.flip(img_data1,1)


    STRM = np.flip(np.moveaxis(np.moveaxis(nib.load(j).get_fdata().astype(np.float32),2,0),1,1),1)
    for g in range(STRM.shape[0]):
        STRM[g,:,:]=np.flipud(STRM[g,:,:])
    stc=np.round(np.array(scipy.ndimage.measurements.center_of_mass(STRM))).astype(int)
    stc =stc - np.array(img_data1.shape) /2
    print(stc)

    ShiftRange=np.round(np.linspace(-20,20,41))
    ShiftIndex=np.random.randint(0,41,3)
    ShiftCoor=np.zeros((1,3))
    ShiftCoor[:,0]=ShiftRange[ShiftIndex[0]]
    ShiftCoor[:,1]=ShiftRange[ShiftIndex[1]]
    ShiftCoor[:,2]=ShiftRange[ShiftIndex[2]]
    ShiftCoor=np.append(np.zeros((1,3)),ShiftCoor,axis=0)
    img_data1=rescale(img_data1.copy(),stc)
#     # img_data1=scipy.ndimage.shift(img_data1,-stc,mode='nearest')
#     STRM1=rescale(STRM.copy(),stc)

    matct=img_data1.astype('float32')
    matct=np.where(matct < -1000, -1000, matct)
    matct=(matct/1000)*0.2+0.2
    matct_rs=matct
    sz=matct_rs.shape

    num_of_projections = 360
    angles = np.linspace(0, 2 * np.pi, num=num_of_projections, endpoint=False)
    angles = angles[[330,0,30]]


    vol_geom = astra.creators.create_vol_geom(sz[1], sz[2],
                                            sz[0])



    for Siter in range(ShiftCoor.shape[0]):
        matct_rs1=rescale(matct_rs.copy(),ShiftCoor[Siter,:])
        # matct_rs1=scipy.ndimage.shift(matct_rs,ShiftCoor[Siter,:],mode='nearest')
        print(['Iteration ' + str(Siter) + ' Shift ' + str(ShiftCoor[Siter,:])])

        phantom_id = astra.data3d.create('-vol', vol_geom, data=matct_rs1)

        SOD = 1000  # [mm]
        ODD = 500  # [mm]
        DS = 1  # [mm]
        ## then each pixel on detector corresponds to SOD/(SOD+ODD) mm of the object
        detector_rows = 512  # Vertical size of detector [pixels].
        detector_cols = 512  # Horizontal size of detector [pixels].

        # phantom_id = astra.data3d.create('-vol', vol_geom, data=matct_rs)

        proj_geomL = \
        astra.create_proj_geom('cone', DS, DS, 512, 512, angles,SOD,ODD)
                                #  (distance_source_origin + distance_origin_detector) /
                                #  detector_pixel_size, 0)

        projections_idL, projectionsL = \
        astra.creators.create_sino3d_gpu(phantom_id, proj_geomL, vol_geom)

        projectionsL = (projectionsL-projectionsL.mean()) / projectionsL.std()
        # projectionsL=np.clip(projectionsL,-3,3)/3
        projectionsL = (projectionsL-projectionsL.min())/(projectionsL.max()-projectionsL.min())


        projcL=np.moveaxis(projectionsL,1,0)
        for i in range(projcL.shape[0]):
            projcL[i,:,:]=np.flipud(projcL[i,:,:])


        Savedir='TestData/'
        for i in range(projcL.shape[0]):
            nimg = nib.Nifti1Image(projcL[i,:,:], np.eye(4))
            filename='P' + str(casenum).zfill(2)+ '_' + CTTYPE + '_' + 'LFOV' + '_' + 'S' + str(Siter).zfill(3) + '_' + str(i)
            nib.save(nimg, Savedir + filename)



        SOD = 700  # [mm]
        ODD = 300  # [mm]
        DS = 0.5  # [mm]
        ## then each pixel on detector corresponds to SOD/(SOD+ODD) mm of the object
        detector_rows = 512  # Vertical size of detector [pixels].
        detector_cols = 512  # Horizontal size of detector [pixels].


        proj_geom = \
        astra.create_proj_geom('cone', DS, DS, 512, 512, angles,SOD,ODD)
                                #  (distance_source_origin + distance_origin_detector) /
                                #  detector_pixel_size, 0)


        # vol_geom = astra.creators.create_vol_geom(sz[1], sz[2],
        #                                         sz[0],stc[1]-100,stc[1]+100, stc[2]-100,stc[2]+100,stc[0]-100,stc[0]+100)
        ## Image Projection


        projections_id, projections = \
        astra.creators.create_sino3d_gpu(phantom_id, proj_geom, vol_geom)

        projections = (projections-projections.mean()) / projections.std()
        # projections=np.clip(projections,-3,3)/3
        projections = (projections-projections.min())/(projections.max()-projections.min())


        projc=np.moveaxis(projections,1,0)
        for i in range(projc.shape[0]):
            projc[i,:,:]=np.flipud(projc[i,:,:])
        for i in range(projc.shape[0]):
            nimg = nib.Nifti1Image(projc[i,:,:], np.eye(4))
            filename='P' + str(casenum).zfill(2)+ '_' + CTTYPE + '_' + 'SFOV' + '_' + 'S' + str(Siter).zfill(3) + '_' + str(i)
            nib.save(nimg, Savedir + filename)

            # astra.data3d.delete(vol_geom)
        astra.data3d.delete(phantom_id)
        # astra.data3d.delete(proj_geom)
        astra.data3d.delete(projections_idL)
        astra.data3d.delete(projections_id)




# sitk.Show(sitk.GetImageFromArray(projc))

pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1


['Case Number is 0']
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
[-6. -3. -9.]
['Iteration 0 Shift [0. 0. 0.]']
['Iteration 1 Shift [-15.   2.  18.]']


pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1


['Case Number is 1']
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
[20. 35. 21.]
['Iteration 0 Shift [0. 0. 0.]']
['Iteration 1 Shift [-20.   7. -13.]']


In [5]:
ShiftCoor

array([[  0.,   0.,   0.],
       [-20.,   7., -13.]])

In [ ]:

# import astra
# import torch
# import torchvision.transforms as transforms

# CTTYPE='CT0'



# data_folder="./DataNii/"
# list_CT=Path(data_folder).glob(('**/*_IMG_' + CTTYPE + '.nii.gz'))
# list_STR=Path(data_folder).glob(('**/*_STR_' + CTTYPE + '.nii.gz'))

# DataInfoTuple=[]
# DataInfoTupleSTR=[]
# n=0
# for i in list_CT:
#     DataInfoTuple.append(i)
#     n=n+1
# for i in list_STR:
#     DataInfoTupleSTR.append(i)
#     n=n+1


# # for i,j in DataInfoTuple:
# for kkk in [0]:

#     i=DataInfoTuple[kkk]
#     j=DataInfoTupleSTR[kkk]

#     s=str(i).find('\P')
#     e=str(i).find('_IMG')
#     casenum=int(str(i)[s+2:e])
#     print(['Case Number is ' + str(casenum)])
#     img_data = nib.load(i).get_fdata()

#     img_data1=np.moveaxis(img_data,2,0)
#     img_data1=np.moveaxis(img_data1,1,2)
#     img_data1=np.flip(img_data1,1)
#     img_data1q=img_data1.copy()
#     # if (kkk==3 or kkk==4 or kkk==7 or kkk==9 or kkk==12 or kkk==13):
#     STRM = np.flip(np.moveaxis(np.moveaxis(nib.load(j).get_fdata().astype(np.float32),2,0),1,1),1)
#     for g in range(STRM.shape[0]):
#         STRM[g,:,:]=np.flipud(STRM[g,:,:])

#     # else:
#     #     STRM = np.flip(np.moveaxis(np.moveaxis(nib.load(j).get_fdata().astype(np.float32),2,0),1,1),0)
#     #     stc=np.round(np.array(scipy.ndimage.measurements.center_of_mass(STRM))).astype(int)


#     stc=np.round(np.array(scipy.ndimage.measurements.center_of_mass(STRM))).astype(int)

#     stc =stc - np.array(img_data1.shape) /2
#     print(stc)
#     img_data1=rescale(img_data1.copy(),stc)
#     # img_data1=scipy.ndimage.shift(img_data1,-stc,mode='nearest')
#     STRM1=rescale(STRM.copy(),stc)

#     # STRM1=scipy.ndimage.shift(STRM,-stc,mode='nearest')

#     matct=img_data1.astype('float32')
#     matct=np.where(matct < -1000, -1000, matct)
#     matct=(matct/1000)*0.2+0.2
#     matct_rs=matct
#     sz=matct_rs.shape

#     num_of_projections = 360
#     angles = np.linspace(0, 2 * np.pi, num=num_of_projections, endpoint=False)

#     vol_geom = astra.creators.create_vol_geom(sz[1], sz[2],
#                                             sz[0])

#     ShiftCoor=np.zeros([1,3])
#     Siter=0
#     # for Siter in range(ShiftCoor.shape[0]):
#     # matct_rs1=scipy.ndimage.shift(matct_rs,ShiftCoor[Siter,:],mode='nearest')
#     matct_rs1=matct_rs
#     print(['Iteration ' + str(Siter) + ' Shift ' + str(ShiftCoor[Siter,:])])

#     phantom_id = astra.data3d.create('-vol', vol_geom, data=matct_rs1)


#     SOD = 700  # [mm]
#     ODD = 300  # [mm]
#     DS = 0.5  # [mm]
#     ## then each pixel on detector corresponds to SOD/(SOD+ODD) mm of the object
#     detector_rows = 512  # Vertical size of detector [pixels].
#     detector_cols = 512  # Horizontal size of detector [pixels].


#     proj_geom = \
#     astra.create_proj_geom('cone', DS, DS, 512, 512, angles,SOD,ODD)
#                             #  (distance_source_origin + distance_origin_detector) /
#                             #  detector_pixel_size, 0)


#     # vol_geom = astra.creators.create_vol_geom(sz[1], sz[2],
#     #                                         sz[0],stc[1]-100,stc[1]+100, stc[2]-100,stc[2]+100,stc[0]-100,stc[0]+100)
#     ## Image Projection


#     projections_id, projections = \
#     astra.creators.create_sino3d_gpu(phantom_id, proj_geom, vol_geom)

#     projections = (projections-projections.mean()) / projections.std()
#     projections=np.clip(projections,-3,3)/3


#     projc=np.moveaxis(projections,1,0)
#     # projc=np.flipud(projc)
#     # nimg = nib.Nifti1Image(projc, np.eye(4))
#     # filename='P' + str(casenum).zfill(2)+ '_' + CTTYPE + '_' + 'SFOV' + '_' + 'S' + str(Siter).zfill(3)
#     # nib.save(nimg, Savedir + filename)

#     phantom_id = astra.data3d.create('-vol', vol_geom, data=STRM1)


#     SOD = 700  # [mm]
#     ODD = 300  # [mm]
#     DS = 0.5  # [mm]
#     ## then each pixel on detector corresponds to SOD/(SOD+ODD) mm of the object
#     detector_rows = 512  # Vertical size of detector [pixels].
#     detector_cols = 512  # Horizontal size of detector [pixels].


#     proj_geom = \
#     astra.create_proj_geom('cone', DS, DS, 512, 512, angles,SOD,ODD)
#                             #  (distance_source_origin + distance_origin_detector) /
#                             #  detector_pixel_size, 0)


#     # vol_geom = astra.creators.create_vol_geom(sz[1], sz[2],
#     #                                         sz[0],stc[1]-100,stc[1]+100, stc[2]-100,stc[2]+100,stc[0]-100,stc[0]+100)
#     ## Image Projection


#     projections_idL, projections = \
#     astra.creators.create_sino3d_gpu(phantom_id, proj_geom, vol_geom)

#     projections = (projections-projections.mean()) / projections.std()
#     projections=np.clip(projections,-3,3)/3


#     projm=np.moveaxis(projections,1,0)
#     # projm=np.flipud(projm)

#         # astra.data3d.delete(vol_geom)
#     astra.data3d.delete(phantom_id)
#     # astra.data3d.delete(proj_geom)
#     astra.data3d.delete(projections_idL)
#     astra.data3d.delete(projections_id)




# # # sitk.Show(sitk.GetImageFromArray(projc))

In [ ]:
sitk.Show(sitk.GetImageFromArray(projc))
interact(display_images_with_alpha_numpy, image_z=(0,projc.shape[0] - 1), alpha=(0.0,1.0,0.05), fixed = fixed(projc), moving=fixed(projm))


In [ ]:
# sitk.Show(sitk.GetImageFromArray(img_data1))
interact(display_images_with_alpha_numpy, image_z=(0,img_data1.shape[0] - 1), alpha=(0.0,1.0,0.05), fixed = fixed(img_data1), moving=fixed(1000*STRM1))


In [ ]:
# interact(display_images_with_alpha_numpy, image_z=(0,projc.shape[0] - 1), alpha=(0.0,1.0,0.05), fixed = fixed(projc), moving=fixed(projcL))


In [ ]:
# # interact(display_images_with_alpha_numpy, image_z=(0,projc.shape[0] - 1), alpha=(0.0,1.0,0.05), fixed = fixed(projc), moving=fixed(projcm))
# ShiftRange=np.round(np.linspace(-20,20,41))
# ShiftIndex=np.random.randint(0,41,300)
# ShiftCoor=np.zeros((100,3))
# ShiftCoor[:,0]=ShiftIndex[0:100]
# ShiftCoor[:,1]=ShiftIndex[100:200]
# ShiftCoor[:,2]=ShiftIndex[200:300]
# ShiftCoor=np.append(np.zeros((1,3)),ShiftCoor,axis=0)
# ShiftRange[ShiftIndex[0:100]]

In [ ]:
data_folder="./DataNii/"
list_CT=Path(data_folder).glob(('**/*_IMG_' + CTTYPE + '.nii.gz'))
list_STR=Path(data_folder).glob(('**/*_STR_' + CTTYPE + '.nii.gz'))

DataInfoTuple=[]
DataInfoTupleSTR=[]
n=0
for i in list_CT:
    DataInfoTuple.append(i)
    n=n+1
for i in list_STR:
    DataInfoTupleSTR.append(i)
    n=n+1


# for i,j in DataInfoTuple:
for kkk in [0]:

    i=DataInfoTuple[kkk]
    j=DataInfoTupleSTR[kkk]

    s=str(i).find('\P')
    e=str(i).find('_IMG')
    casenum=int(str(i)[s+2:e])
    print(['Case Number is ' + str(casenum)])
    img_data = nib.load(i).get_fdata()

    img_data1=np.moveaxis(img_data,2,0)
    img_data1=np.moveaxis(img_data1,1,2)
    img_data1=np.flip(img_data1,1)
    img_data1q=img_data1.copy()
    # if (kkk==3 or kkk==4 or kkk==7 or kkk==9 or kkk==12 or kkk==13):
    STRM = np.flip(np.moveaxis(np.moveaxis(nib.load(j).get_fdata().astype(np.float32),2,0),1,1),1)
    for g in range(STRM.shape[0]):
        STRM[g,:,:]=np.flipud(STRM[g,:,:])

    # else:
    #     STRM = np.flip(np.moveaxis(np.moveaxis(nib.load(j).get_fdata().astype(np.float32),2,0),1,1),0)
    #     stc=np.round(np.array(scipy.ndimage.measurements.center_of_mass(STRM))).astype(int)


    stc=np.round(np.array(scipy.ndimage.measurements.center_of_mass(STRM))).astype(int)

    stc =stc - np.array(img_data1.shape) /2

In [ ]:
img_data1qt=torch.from_numpy(img_data1q)
astc=np.abs(np.round(stc)).astype(int)
istc=tuple(np.round(-stc).astype(int))

img_data1qt=torch.nn.functional.pad(img_data1qt.unsqueeze(0).unsqueeze(0),(0,astc[2], 0,astc[1], 0,astc[0]),mode='replicate')
img_data1qt=torch.roll(img_data1qt, shifts=istc, dims=(2,3,4) )
img_data1qt=img_data1qt[:,:,0:492,0:577,0:577].squeeze(0).squeeze(0).numpy()
stc
print([img_data1.shape , img_data1qt.shape])

In [ ]:
img_data1qn=scipy.ndimage.shift(img_data1q,np.array(istc),mode='nearest')

In [ ]:
interact(display_images_with_alpha_numpy, image_z=(0,img_data1qt.shape[0] - 1), alpha=(0.0,1.0,0.05), fixed = fixed(img_data1qt), moving=fixed(img_data1qn))
